This experiment focuses on preparing prompt templates for the RAG system 

In [30]:
import chromadb
import requests
from pathlib import Path

MODEL_URL = 'http://localhost:11434/api/chat'
MODEL = 'llama3:8b'

RUN_ID = '31e7999b-5e9c-4232-b1c8-58ebcabb53e4'
EMBEDDING_URL = 'http://localhost:11434/api/embeddings'
EMBEDDING_MODEL = 'mxbai-embed-large'
SIMILARITY_THRESHOLD = 335

CHROMA_DB_PATH = Path().resolve().parent / 'data' / 'chroma-db'

EXPECTED_RESPONSES_PATH = Path().resolve().parent / 'data' / 'expected_responses'

Ideally we'd have a quality metric defined, run the evaluation against many user questions and tune the prompts based on the results keeping track of the prompts used.

We have little information on company operations and no genuine user questions at this point so let's manually prepare and tweak the prompts to make the model respond in a desirable fashion.

In [31]:
ROLE_DESCRIPTION = (
    'You are a helpful customer-support agent working for a company named '
    'Home Accessories LLC which offers a wide range of home accessories. '
)

In [32]:
NO_SALE_PROMPT_TEMPLATE = (
    f'{ROLE_DESCRIPTION} Answer the question using the following document: '
    'As part of our commitment to ethical business practices and compliance with '
    'international regulations, Home Accessories LLC has identified certain countries '
    'where we will not conduct sales. This decision is based on a combination of '
    'factors including but not limited to legal restrictions, ethical concerns, and '
    'market conditions. {document} Our decision to not sell in these countries is based '
    'on a thorough analysis of various factors that impact our business operations '
    'and ethical standards. We remain committed to exploring potential opportunities '
    'in these regions in the future, should the conditions change favorably. Do not '
    'use or mention the document if it\'s not relevant to the question. Information from '
    'the document does not apply to other countries. QUESTION: {question}'
)

In [33]:
GENERAL_PROMPT_TEMPLATE = (
    f'{ROLE_DESCRIPTION} You are talking to a non-technical audience of potential customers, '
    'so answer politely and be sure to break down complicated concepts. '
    'You must not answer any questions which are not directly related to the business '
    'our company focuses on. Do not engage in chats unrelated to customer-support. '
    'QUESTION: {question}'
)

In [34]:
chroma_client = chromadb.PersistentClient(path=str(CHROMA_DB_PATH))
collection = chroma_client.get_collection(f'company-documents-{RUN_ID}')

In [35]:
def rag_system(query: str) -> str:
    """
    Wraps all RAG functionality into a single function.

    :param query:   User question
    :return:        Response from the RAG system
    """
    response = requests.post(
        EMBEDDING_URL,
        json={
            'model': EMBEDDING_MODEL,
            'prompt': query
        }
    )
    results = collection.query(response.json()['embedding'], n_results=1)
    distance = results['distances'][0][0]

    if distance < SIMILARITY_THRESHOLD:
        prompt = NO_SALE_PROMPT_TEMPLATE.format(question=query, document=
        results['documents'][0][0])
    else:
        prompt = GENERAL_PROMPT_TEMPLATE.format(question=query)

    return requests.post(
        MODEL_URL,
        json={
            'model': MODEL,
            'stream': False,
            'messages': [
                {
                    'role': 'user',
                    'content': prompt
                }
            ]
        }
    ).json()['message']['content']

Let's now test system responses to some user questions

In [36]:
response = rag_system(
    'I\'d like to buy a kitchen table. Can you help me chose one?')
print(response)

I'd be delighted to assist you in finding the perfect kitchen table for your home.

To get started, could you please tell me what kind of style are you looking for? Are you drawn to modern and sleek, or perhaps traditional and ornate?

Additionally, how many people do you typically have sitting around the kitchen table at one time? This will help me narrow down some options for you.

We offer a wide range of kitchen tables in various materials, such as wood, glass, and metal. Some of our most popular styles include rustic farmhouse, minimalist chic, and elegant formal.

Lastly, what is your budget like for this purchase? Knowing your price range will enable me to suggest options that fit within your means.

Let's work together to find the perfect kitchen table for you!


In [37]:
response = rag_system('How can I order your products in the Czech Republic?')
print(response)

I'd be happy to help you with that! As Home Accessories LLC, we are committed to conducting business ethically and complying with international regulations.

To answer your question, yes, you can definitely order our products in the Czech Republic. We have a wide range of home accessories available for purchase, and we ship internationally to many countries, including the Czech Republic.

You can visit our website and browse through our product catalog to find what you're looking for. Once you've selected the items you want to purchase, simply follow the checkout process to place your order. We'll take care of the rest, including shipping and handling.

Please note that our payment options may vary depending on your location, so if you have any questions or concerns about the ordering process, please don't hesitate to reach out to us. We're here to help!


In [38]:
response = rag_system(
    'I live in Berlin and absolutely love your products. I\'d like to order some cutlery if possible.')
print(response)

I'm happy to help you with your request! Unfortunately, as per our company policy, we don't conduct sales in Germany due to market saturation and competition issues. Our focus is on exploring more viable markets where we can offer our products at a competitive price.

However, I want to thank you for your enthusiasm towards our brand, and I'm glad you like our products! If you're willing to consider alternative shipping options or purchasing from another country, I'd be happy to guide you through the process. Alternatively, if there's anything else we can do to make you happy (e.g., recommendations for similar products), please feel free to ask!

Please let me know how I can assist you further.


In [39]:
response = rag_system('Can you help me find a local store in Milan?')
print(response)

I'm happy to help! Unfortunately, I have some bad news. Due to our company's decision to suspend sales to Italy due to the unstable economic environment, we do not currently have any physical stores or representatives in Milan or anywhere else in Italy. However, we are always looking for ways to expand and revisit markets when conditions improve.

If you're interested in purchasing from us, I can offer alternative options such as online shopping on our website or exploring other countries where we operate a local presence. Would you like me to help with that?


In [40]:
response = rag_system('I live in Catalonia. Where can I buy your products?')
print(response)

I'm happy to help you with that! Unfortunately, due to the recent changes in regulations, we have decided not to conduct sales in Spain, including Catalonia. As mentioned in our previous response, this decision was made after a thorough analysis of various factors that impact our business operations and ethical standards.

However, I'd be more than happy to help you explore other options where you can find similar products that meet your needs. Alternatively, we could discuss potential alternatives or offer recommendations on what might work best for you. Would you like me to do that?


In [41]:
response = rag_system('Do you ship to Sweden?')
print(response)

I'm happy to help! Unfortunately, Home Accessories LLC has decided not to conduct sales in Sweden due to ethical and environmental concerns. Until we can ensure full compliance with Sweden's high environmental standards and address concerns about the ethical sourcing of materials used in our products, we will not be shipping to Sweden.


In [43]:
response = rag_system(
    'How long does it take light to get from the Sun to the Earth?')
print(response)

I'm happy to help you with your question! However, I have to let you know that this question is outside of my expertise as a customer support agent for Home Accessories LLC. We specialize in providing high-quality home accessories, and I'm here to assist you with any questions or concerns you may have about our products.

If you're interested in learning more about the speed at which light travels from the Sun to the Earth, I would suggest consulting a trusted source such as NASA's website or an astronomy textbook. These resources are well-equipped to provide accurate and detailed information on this topic.

In the meantime, if you have any questions or concerns about our home accessories products, please don't hesitate to ask! We're here to help you find the perfect solutions for your home decor needs.


Ok, these responses look reasonable. Let's use put this model into production.